In [1]:
import turicreate
import math
import string

In [2]:
# Read CSV files

whp_1 = turicreate.SFrame.read_csv('Congressional District_1.csv', verbose = False)
whp_2 = turicreate.SFrame.read_csv('County_2.csv', verbose = False)
whp_3 = turicreate.SFrame.read_csv('ZIP Code_3.csv', verbose = False)
whp_4 = turicreate.SFrame.read_csv('Tract_4.csv', verbose = False)
whp_5 = turicreate.SFrame.read_csv('Block Group_5.csv', verbose = False)

print((whp_1.num_rows(), whp_1.num_columns()))
print((whp_2.num_rows(), whp_2.num_columns()))
print((whp_3.num_rows(), whp_3.num_columns()))
print((whp_4.num_rows(), whp_4.num_columns()))
print((whp_5.num_rows(), whp_5.num_columns()))

(433, 48)
(3108, 48)
(31808, 47)
(72247, 47)
(215752, 47)


In [3]:
# Set ID features to join predicted results into mappable Geodatabase format

ID_features = ['OBJECTID', 'ID']

# Set features for Logistic Classifier

features = ['2020 Total Population (Esri)',
            '2020 Population Density (Pop per Square Mile) (Esri)',
            '2020 Total Households (Esri)',
            '2020 Total Housing Units (Esri)',
            '2020 Owner Occupied Housing Units (Esri)',
            '2020 Renter Occupied Housing Units (Esri)',
            '2010-2020 Population: Annual Growth Rate (Esri)',
            '2020 Housing Affordability Index (Esri)',
            '2020 Percent of Income for Mortgage (Esri)',
            '2010 Total Population (Esri 2020)',
            '2010 Total Households (Esri 2020)',
            '2010 Total Housing Units (Esri 2020)',
            '2020 Minority Population (Esri)',
            '2020 Diversity Index (Esri)',
            '2020 Median Household Income (Esri)',
            '2020 Per Capita Income (Esri)',
            '2020 Median Home Value (Esri)',
            '2020 Average Home Value (Esri)',
            '2025 Total Population (Esri)',
            '2025 Population Density (Pop per Square Mile) (Esri)',
            '2025 Total Households (Esri)',
            '2025 Total Housing Units (Esri)',
            '2020-2025 Population: Annual Growth Rate (Esri)',
            '2000-2010 Population: Annual Growth Rate (U.S. Census)',
            'Annualized Population Change 2010-2020',
            '% Population Change between 2010-2020',
            'Annualized Housing Unit Change 2010-2020',
            '% Housing Unit Change between 2010-2020',
            'Annualized Household Change 2010-2020',
            '% Household Change between 2010-2020',
            # 'Population Weighted Wildfire Hazard Potential Score',
            'Lowest Possible Wildfire Hazard Potential Score',
            'Highest Possible Wildfire Hazard Potential Score',
            'Difference between MIN and MAX Wildfire Hazard Potential Scores',
            'Standard Deviation of Wildfire Hazard Potential Score',
            'Sum of all Wildfire Hazard Potential Scores within Area',
            'Number of Unique Wildfire Potential Scores',
            'Most Frequently Occurring Wildfire Hazard Potential Score',
            'Least Frequently Occurring Wildfire Hazard Potential Score',
            'Median Wildfire Hazard Potential Score',
            'Shape__Area',
            'Shape__Length']
print(len(features))

# Set target for Logistic Classifier

whp_target = 'Average Wildfire Hazard Potential Score'

41


In [4]:
# Select ID features, model features and model target from data frames

whp_1_select = whp_1.select_columns(features + [whp_target])
whp_2_select = whp_2.select_columns(ID_features + features + [whp_target])
whp_3_select = whp_3.select_columns(features + [whp_target]) 
whp_4_select = whp_4.select_columns(features + [whp_target])
whp_5_select = whp_5.select_columns(features + [whp_target])
whp = whp_1_select + whp_3_select + whp_4_select + whp_5_select
whp_test = whp_2_select # County_2

print((whp.num_rows(), whp.num_columns()))
print((whp_test.num_rows(), whp_test.num_columns()))

(320240, 42)
(3108, 44)


In [5]:
def risk_reclassification(average_whp) :
    if average_whp < 1.0 :
        return -1 # Low Risk
    if average_whp >= 3.0 :
        return +1 # High Risk
    else :
        return +0 # Medium Risk

# Reclassify risk
    
whp['risk'] = whp[whp_target].apply(risk_reclassification)
whp_test['risk'] = whp_test[whp_target].apply(risk_reclassification)
target = 'risk'

In [6]:
# Drop observations with missing values in training data

whp, whp_with_na = whp[features + [whp_target] + [target]].dropna_split()

num_rows_with_na = whp_with_na.num_rows()
num_rows = whp.num_rows()

print('whp Dropping %s observation, keeping %s' % (num_rows_with_na, num_rows))

whp Dropping 15305 observation, keeping 304935


In [7]:
# Drop observations with missing values in testing data

whp_test, whp_test_with_na = whp_test[ID_features + features + [whp_target] + [target]].dropna_split()

num_rows_with_na = whp_test_with_na.num_rows()
num_rows = whp_test.num_rows()

print('whp_test Dropping %s observation, keeping %s' % (num_rows_with_na, num_rows))

whp_test Dropping 3 observation, keeping 3105


In [8]:
whp_test_with_na.print_rows(num_rows = 3, num_columns = 3)

+----------+-------+------------------------------+-----+
| OBJECTID |   ID  | 2020 Total Population (Esri) | ... |
+----------+-------+------------------------------+-----+
|   2620   | 48261 |             445              | ... |
|   2624   | 48269 |             270              | ... |
|   2640   | 48301 |              88              | ... |
+----------+-------+------------------------------+-----+
[3 rows x 45 columns]



In [9]:
# train_data, test_data = whp.random_split(0.8, seed = 1)
train_data = whp
test_data = whp_test
print(len(train_data))
print(len(test_data))

# Calculate the Majority Classifier accuracies in training and testing datasets
# Models built should at least have better performances than Majority Classifiers

def majority_classifier(data) :
    low = 0
    medium = 0
    high = 0
    results = data['risk']
    for i in range(len(results)) :
        if results[i] == -1 :
            low += 1
        if results[i] == 0 :
            medium += 1
        if results[i] == +1 :
            high += 1
    return float(max(low, medium, high)) / len(results)

print('Train Data Majority Classifier Accuracy: ' + str(majority_classifier(train_data)))
print('Test Data Majority Classifier Accuracy: ' + str(majority_classifier(test_data)))

304935
3105
Train Data Majority Classifier Accuracy: 0.7226326922130946
Test Data Majority Classifier Accuracy: 0.527536231884058


In [10]:
# Define maximum iteration

# iterations = [5, 10, 15, 20, 30, 50, 100, 200, 300, 500]
iterations = [5, 10, 15, 20, 25, 30, 40, 50, 100, 200]
# iterations = [5, 200]
lg_train_accuracies = list()
lg_test_accuracies = list()

In [11]:
# Train and Test Logistic Classifiers

for iteration in iterations :
    lg_model = turicreate.logistic_classifier.create(dataset = train_data,
                                                     target = target, 
                                                     features = features,
                                                     validation_set = None,
                                                     max_iterations = iteration,
                                                     l1_penalty = 0.1,
                                                     l2_penalty = 0.1,
                                                     verbose = False)
    train_accuracy = lg_model.evaluate(train_data, metric = 'accuracy')['accuracy']
    test_accuracy = lg_model.evaluate(test_data, metric = 'accuracy')['accuracy']
    # print((train_accuracy, test_accuracy))
    print('Train Accuracy: ' + str(train_accuracy) + '   Test Accuracy: ' + str(test_accuracy))
    lg_train_accuracies.append(train_accuracy)
    lg_test_accuracies.append(test_accuracy)

Train Accuracy: 0.8713889845376883   Test Accuracy: 0.7903381642512077
Train Accuracy: 0.8875170118221916   Test Accuracy: 0.8099838969404187
Train Accuracy: 0.8993293652745667   Test Accuracy: 0.8225442834138487
Train Accuracy: 0.9073704232049453   Test Accuracy: 0.8312399355877617
Train Accuracy: 0.9147785593651105   Test Accuracy: 0.8466988727858293
Train Accuracy: 0.9212127174643776   Test Accuracy: 0.856682769726248
Train Accuracy: 0.9291094823486973   Test Accuracy: 0.8676328502415459
Train Accuracy: 0.9333136570088707   Test Accuracy: 0.8756843800322062
Train Accuracy: 0.9423877219735354   Test Accuracy: 0.8882447665056361
Train Accuracy: 0.9475953891813009   Test Accuracy: 0.9014492753623189


In [12]:
# Look into the Logistic Classifier

lg_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 84
Number of examples             : 304935
Number of classes              : 3
Number of feature columns      : 41
Number of unpacked features    : 41

Hyperparameters
---------------
L1 penalty                     : 0.1
L2 penalty                     : 0.1

Training Summary
----------------
Solver                         : fista
Solver iterations              : 200
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 150.8954

Settings
--------
Log-likelihood                 : 37481.7006

Highest Positive Coefficients
-----------------------------
Lowest Possible Wildfire Hazard Potential Score : 16.5079
Lowest Possible Wildfire Hazard Potential Score : 14.7062
Standard Deviation of Wildfire Hazard Potential Score : 5.8699
Median Wildfire Hazard Potential Score : 5.5104
Most Frequently Occurring Wildfire Hazard Potential Score : 3.5034

L

In [13]:
whp_test['predict_risk'] = lg_model.predict(whp_test)
whp_test_with_na['predict_risk'] = 5
whp_test_with_na.print_rows(num_rows = 3, num_columns = 46)
whp_test = whp_test + whp_test_with_na

+----------+-------+------------------------------+-------------------------------+
| OBJECTID |   ID  | 2020 Total Population (Esri) | 2020 Population Density (P... |
+----------+-------+------------------------------+-------------------------------+
|   2620   | 48261 |             445              |              0.3              |
|   2624   | 48269 |             270              |              0.3              |
|   2640   | 48301 |              88              |              0.1              |
+----------+-------+------------------------------+-------------------------------+
+------------------------------+-------------------------------+-------------------------------+
| 2020 Total Households (Esri) | 2020 Total Housing Units (... | 2020 Owner Occupied Housin... |
+------------------------------+-------------------------------+-------------------------------+
|             157              |              233              |               45              |
|             108       

In [14]:
# whp_test.print_rows(num_rows = 5, num_columns = 46)
whp_test.print_rows(num_rows = 3, num_columns = 3)

+----------+------+------------------------------+-----+
| OBJECTID |  ID  | 2020 Total Population (Esri) | ... |
+----------+------+------------------------------+-----+
|    1     | 1001 |            58224             | ... |
|    2     | 1003 |            227660            | ... |
|    3     | 1005 |            26326             | ... |
+----------+------+------------------------------+-----+
[3108 rows x 46 columns]



In [15]:
county_predict = whp_test.select_columns(ID_features + ['risk'] + ['predict_risk'])
county_predict.print_rows(num_rows = 5)

+----------+------+------+--------------+
| OBJECTID |  ID  | risk | predict_risk |
+----------+------+------+--------------+
|    1     | 1001 |  0   |      0       |
|    2     | 1003 |  0   |      0       |
|    3     | 1005 |  0   |      0       |
|    4     | 1007 |  0   |      0       |
|    5     | 1009 |  0   |      0       |
+----------+------+------+--------------+
[3108 rows x 4 columns]



In [16]:
# Export the predicted results to CSV file format

county_predict.export_csv('Model_Predict_Libo.csv')